In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from local.torch_basics import *
from local.test import *
from local.basics import *
from local.data.all import *
from local.vision.core import *
from local.notebook.showdoc import show_doc
from local.audio.core import *
from local.audio.augment import *
from local.vision.learner import *
from local.vision.models.xresnet import *
from local.metrics import *
from local.callback.schedule import *
import torchaudio
from fastprogress import progress_bar as pb, master_bar
from numba import njit, prange
from rocket import ROCKET, ROCKETMAX
from sklearn.linear_model import RidgeClassifierCV

In [ ]:
PATH_25 = Config()['data_path'] / '25_speakers'
x = AudioGetter("", recurse=True, folders=None)
files_25  = x(PATH_25)
oa = OpenAudio(files_25)

In [ ]:
files_25

(#3417) [/home/jupyter/.fastai/data/25_speakers/id08870/u-6MoWmZzeU/00081.wav,/home/jupyter/.fastai/data/25_speakers/id08870/_62vf-iMaVM/00061.wav,/home/jupyter/.fastai/data/25_speakers/id08870/_62vf-iMaVM/00062.wav,/home/jupyter/.fastai/data/25_speakers/id08870/_62vf-iMaVM/00060.wav,/home/jupyter/.fastai/data/25_speakers/id08870/RyzxIgw3NHQ/00050.wav,/home/jupyter/.fastai/data/25_speakers/id08870/RyzxIgw3NHQ/00036.wav,/home/jupyter/.fastai/data/25_speakers/id08870/RyzxIgw3NHQ/00038.wav,/home/jupyter/.fastai/data/25_speakers/id08870/RyzxIgw3NHQ/00046.wav,/home/jupyter/.fastai/data/25_speakers/id08870/RyzxIgw3NHQ/00035.wav,/home/jupyter/.fastai/data/25_speakers/id08870/RyzxIgw3NHQ/00053.wav...]

In [ ]:
def AudioBlock(cls=AudioItem): return TransformBlock(type_tfms=cls.create, batch_tfms=IntToFloatTensor)

In [ ]:
def mn(x): return round(sum(x)/len(x), 3)

In [ ]:
def extract_data(dbunch, mb):
    train_dl, valid_dl = dbunch
    sigs_train, labels_train = [],[]
    for batch in pb(train_dl, parent=mb):
        sigs_train.append(batch[0])
        labels_train.append(batch[1])
    sigs_valid, labels_valid = [],[]
    for batch in pb(valid_dl, parent=mb):
        sigs_valid.append(batch[0])
        labels_valid.append(batch[1])
    sigs_train = torch.cat(sigs_train, axis=0).cpu().numpy()
    labels_train = torch.cat(labels_train, axis=0).cpu().numpy()
    sigs_valid = torch.cat(sigs_valid, axis=0).cpu().numpy()
    labels_valid = torch.cat(labels_valid, axis=0).cpu().numpy()
    return sigs_train, labels_train, sigs_valid, labels_valid

In [ ]:
def fit_n_kernels(n_kernels, datapath,norm,runs=1, rocket_max=False):
    get_audio = AudioGetter("", recurse=True, folders=None)
    files = get_audio(datapath)
    scores, times = [],[]
    mb = master_bar(range(runs))
    for i in mb:
        start = time.time()
        if rocket_max: rocket = ROCKETMAX(1, 32000, n_kernels=n_kernels, kss=[7, 9, 11], stride=5).to(device)
        else         : rocket = ROCKET(1, 32000, n_kernels=n_kernels, kss=[7, 9, 11], stride=5).to(device)
        tfms = [CropSignal(2000), norm]
        gpu_tfms = [Cuda(), rocket]
        dbunch = auds.databunch(datapath, item_tfms=tfms,  batch_tfms=gpu_tfms, bs=128)
        x_train, y_train, x_valid, y_valid = extract_data(dbunch, mb)
        x_train, x_valid = map(lambda x: x.squeeze(1), (x_train, x_valid))
        classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 7), normalize=True)
        classifier.fit(x_train, y_train)
        scores.append(classifier.score(x_valid, y_valid))
        times.append(round(time.time() - start, 3))
    return mn(scores), mn(times)

In [ ]:
def GetSig():
    def _inner(ai:AudioItem)->AudioItem:
        return ai.sig
    return _inner

In [ ]:
def Normalize(m, s):
    def _inner(ai:AudioItem)->AudioItem:
        sig = ai.sig.clone()
        norm_sig = (sig - m)/(s + 1e-8)
        return AudioItem((norm_sig, ai.sr, ai.path))
    return _inner

In [ ]:
def get_normalizer(datapath):
    get_audio = AudioGetter("", recurse=True, folders=None)
    files = get_audio(datapath)
    total_mean = total_std = 0
    oa = OpenAudio(files)
    num_files=len(files)
    for i in pb(range(num_files), leave=False):
        total_mean += oa(i).sig.mean()
        total_std  += oa(i).sig.std()
    mean = total_mean/num_files
    std  = total_std/num_files
    return Normalize(mean, std)

In [ ]:
auds = DataBlock(blocks=(AudioBlock, CategoryBlock),  
                 get_items=get_audio_files, 
                 splitter=RandomSplitter(),
                 get_y=lambda x: str(x).split('/')[-3][3:])

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [ ]:
norm = get_normalizer(PATH_25)

In [ ]:
fit_n_kernels(1, PATH_25, norm, runs=10)

(0.148, 2.444)

In [ ]:
fit_n_kernels(1, PATH_25, norm, runs=10, rocket_max=True)

(0.173, 2.573)

In [ ]:
fit_n_kernels(10, PATH_25, norm, runs=10)

(0.274, 2.553)

In [ ]:
fit_n_kernels(10, PATH_25, norm, runs=10, rocket_max=True)

(0.422, 2.665)

In [ ]:
fit_n_kernels(100, PATH_25, norm, runs=10)

(0.54, 4.711)

In [ ]:
fit_n_kernels(100, PATH_25, norm, runs=10, rocket_max=True)

(0.674, 6.893)

In [ ]:
fit_n_kernels(1000, PATH_25, norm, runs=1)

(0.631, 33.178)

In [ ]:
fit_n_kernels(1000, PATH_25, norm, runs=1, rocket_max=True)

(0.794, 49.849)

In [ ]:
fit_n_kernels(5000, PATH_25, norm, runs=1)

(0.738, 155.002)

In [ ]:
fit_n_kernels(5000, PATH_25, norm, runs=1, rocket_max=True)

(0.798, 246.696)

In [ ]:
fit_n_kernels(10000, PATH_25, norm, runs=1)

(0.78, 303.796)

In [ ]:
fit_n_kernels(10000, PATH_25, norm, runs=1, rocket_max=True)

(0.832, 527.932)

# Applying the same code to 11 speaker problem

In [ ]:
PATH_11 = Config()['data_path'] / '11_speakers'
x = AudioGetter("", recurse=True, folders=None)
files_11  = x(PATH_11)
oa = OpenAudio(files_11)
norm = Normalize(-3e-06, 0.0492)

In [ ]:
files_11

(#1916) [/home/jupyter/.fastai/data/11_speakers/id09052/GLB6siMkVP8/00344.wav,/home/jupyter/.fastai/data/11_speakers/id09052/GLB6siMkVP8/00199.wav,/home/jupyter/.fastai/data/11_speakers/id09052/GLB6siMkVP8/00233.wav,/home/jupyter/.fastai/data/11_speakers/id09052/GLB6siMkVP8/00323.wav,/home/jupyter/.fastai/data/11_speakers/id09052/GLB6siMkVP8/00266.wav,/home/jupyter/.fastai/data/11_speakers/id09052/GLB6siMkVP8/00241.wav,/home/jupyter/.fastai/data/11_speakers/id09052/GLB6siMkVP8/00203.wav,/home/jupyter/.fastai/data/11_speakers/id09052/GLB6siMkVP8/00276.wav,/home/jupyter/.fastai/data/11_speakers/id09052/GLB6siMkVP8/00242.wav,/home/jupyter/.fastai/data/11_speakers/id09052/GLB6siMkVP8/00245.wav...]

In [ ]:
fit_n_kernels(1000, PATH_11)

TypeError: object of type 'PosixPath' has no len()

In [ ]:
fit_n_kernels(1000, PATH_11, rocket_max=True)

(0.9164490861618799, 29.327)

## Lastly look at 1 kernel prediction power for 11 speakers

In [ ]:
def fit_n_kernels(n_kernels, datapath, rocket_max=False):
    mn, std = get_mean_and_std(datapath)
    norm = Normalize(mn, std)
    start = time.time()
    if rocket_max: rocket = ROCKETMAX(1, 32000, n_kernels=n_kernels, kss=[7, 9, 11], stride=5).to(device)
    else         : rocket = ROCKET(1, 32000, n_kernels=n_kernels, kss=[7, 9, 11], stride=5).to(device)
    gpu_tfms = [Cuda(), rocket]
    dbunch = auds.databunch(datapath, item_tfms=tfms,  batch_tfms=gpu_tfms, bs=128)
    x_train, y_train, x_valid, y_valid = extract_data(dbunch)
    x_train, x_valid = map(lambda x: x.squeeze(1), (x_train, x_valid))
    classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 7), normalize=True)
    classifier.fit(x_train, y_train)
    score = classifier.score(x_valid, y_valid)
    t = round(time.time() - start, 3)
    return score, t

In [ ]:
scores, times = [],[]
for i in range(50):
    s,t = fit_n_kernels(1, PATH_11)
    scores.append(s)
    times.append(t)
print("Avg score no rocketmax", mn(scores))
print("Avg time no rocketmax", mn(times))

Avg score no rocketmax 0.288
Avg time no rocketmax 1.767


In [ ]:
scores, times = [],[]
for i in range(10):
    s,t = fit_n_kernels(1, PATH_11, rocket_max=True)
    scores.append(s)
    times.append(t)
print("Avg score w rocketmax", mn(scores))
print("Avg time w rocketmax", mn(times))

KeyError: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/anaconda3/envs/dev/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 178, in _worker_loop
    data = fetcher.fetch(index)
  File "/opt/anaconda3/envs/dev/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 34, in fetch
    data = next(self.dataset_iter)
  File "/home/jupyter/rob/dev_fastai/dev/local/data/load.py", line 100, in create_batches
    yield from map(self.do_batch, self.chunkify(res))
  File "/home/jupyter/rob/dev_fastai/dev/local/core/utils.py", line 234, in chunked
    res = list(itertools.islice(it, cs))
  File "/home/jupyter/rob/dev_fastai/dev/local/data/load.py", line 113, in do_item
    try: return self.after_item(self.create_item(s))
  File "/home/jupyter/rob/dev_fastai/dev/local/data/load.py", line 119, in create_item
    def create_item(self, s):  return next(self.it) if s is None else self.dataset[s]
  File "/home/jupy